In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

def clean(text):
    text = re.sub(r'\|\|\|', r' ', text)
    text = text.replace('„','')
    text = text.replace('“','')
    text = text.replace('"','')
    text = text.replace('\'','')
    text = text.replace('-',' ')
    text = text.replace('&', 'and')
    text = text.lower()
    return text

def remove_stopwords(text):
    engl_stops = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in engl_stops])

def tokenize(text):
    return list(filter(lambda word : len(word) > 3, word_tokenize(text)))

tagged_descs = []
cmp_classes = []
total = 0
for i, cls_arr in enumerate(subj_classes.values()):
    print(total, end='\r')
    for dep, code, title, desc in cls_arr:
        if desc and len(desc) > 100:
            cmp_classes.append((dep, code, title, desc))
            tagged_descs.append(TaggedDocument(words=tokenize(remove_stopwords(clean(title + " " + desc))), tags=[total]))
            total += 1

NameError: name 'subj_classes' is not defined

In [2]:
from gensim.models.callbacks import CallbackAny2Vec
import multiprocessing

class VecCallback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        self.epoch += 1
        print(f"Starting epoch {self.epoch}")
        
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print(f'Loss after epoch {self.epoch}: {loss}')
        
cores = multiprocessing.cpu_count()
vec_model = Doc2Vec(dm=0, vector_size=50, alpha=0.025, workers=cores)
print("Initialized model")
vec_model.build_vocab(tagged_descs)
print("Built vocab")
vec_model.train(tagged_descs, total_examples=vec_model.corpus_count, epochs=50, callbacks=[VecCallback()])

Initialized model
Built vocab


RuntimeError: you must first build vocabulary before training the model

In [3]:
def preprocess(text):
    text = str(filter(lambda c: c.isalnum(), list(text)))
    return tokenize(remove_stopwords(clean(text)))

def gen_recs(inp, num_recs):
    print(preprocess(inp))
    inf = vec_model.infer_vector(preprocess(inp))
    sims = vec_model.dv.most_similar([inf], topn=100)
    titles = set()
    satisfied = 0
    for i, sim in sims:
        course = cmp_classes[i]
        if course[2] not in titles:
            titles.add(course[2])
            satisfied += 1
            print(sim)
            print(course)
            if satisfied == num_recs:
                break
    print(len(sims))

print("=" * 10)
gen_recs('Object Oriented Programming', 10)
print("=" * 10)
gen_recs('puzzles and paradoxes philosophy', 10)

['filter', 'object', '0x1047503d0']


KeyError: 'sample_int'

In [4]:
vec_model.save('course_embeddings.model')
gen_recs("""Our Engineering team within our Technology organization is building the future of home and looking for dynamic, hardworking and talented students to join us for a 6-month Co-op from January 9, 2023 - June 23, 2023. As a Software Engineering Co-op you will dive into a number of complex technical projects and gain a better understanding of our business. The Co-op program is designed to give you real work on your team, so you’ll be contributing to the goals of your team and making an impact. You will have the opportunity to work within one of the following groups:""", 10)

NameError: name 'json' is not defined